In [8]:
import numpy as np 
import pandas as pd 
import gmaps 
import gmaps.datasets 
import folium
from folium.plugins import HeatMap

In [9]:
EVD=pd.read_csv("data.csv") 


In [10]:
EVD.head()

,Sample ID,PatientNo,Age,DOB_Year,Year of enrollment,Date of hospital entry,AdminDate,SiteRecruitment,Gender,DateOnset,...,Parechovirus,Picobirnavirus,Porprismacovirus,Protoparvovirus,Rotavirus,Rubulavirus,Salivirus,Sapovirus,Unclassified virus,is_coinf
0,16370x1,04VZ-10-0003,89,1923,2012,31/10/2012,01/11/2012,4,2,31/10/2012,...,2,2,2,2,2,2,2,2,2,NaN
1,16370x2,04VZ-10-0004,1,2011,2012,01/11/2012,01/11/2012,4,2,29/10/2012,...,2,2,2,2,2,2,2,2,2,3.0
2,16370x3,04VZ-10-0005,29,1983,2012,02/11/2012,02/11/2012,4,2,01/11/2012,...,2,2,2,2,2,2,2,2,2,NaN
3,16370x4,04VZ-10-0007,0,2012,2012,09/11/2012,09/11/2012,4,1,08/11/2012,...,2,2,2,2,2,2,2,2,2,NaN
4,16370x5,04VZ-10-0009,1,2011,2012,15/11/2012,15/11/2012,4,1,15/11/2012,...,2,2,2,2,2,2,2,2,2,1.0


In [12]:
EVD_common = pd.read_csv("data_common.csv") 

num_c = len(EVD_common)

lat_c = np.array(EVD_common["LATITUDE"][0:num])                        # get lat
lon_c = np.array(EVD_common["LONGITUDE"][0:num])                       # get lon


data1_c = [[lat_c[i],lon_c[i]] for i in range(num_c)]     #make the data in the form of [lats,lons,weights]
map_common = folium.Map(location=[1,2],zoom_start=7)    #draw the map
HeatMap(data1_c).add_to(map_common)  # draw the heatmap to the map we created

map_common

In [13]:
EVD_uncommon = pd.read_csv("data_uncommon.csv") 

num_uc = len(EVD_uncommon)

lat_uc = np.array(EVD_uncommon["LATITUDE"][0:num])                       #lat
lon_uc = np.array(EVD_uncommon["LONGITUDE"][0:num])                      #lon  


data1_uc = [[lat_uc[i],lon_uc[i]] for i in range(num_uc)]    #make the data in the form of [lats,lons,weights]
map_uncommon = folium.Map(location=[1,2],zoom_start=7)     #draw the map
HeatMap(data1_uc).add_to(map_uncommon) # draw the heatmap to the map we created
map_uncommon

In [37]:
from sklearn.kernel_ridge import KernelRidge
import seaborn as sns
import sklearn


impact = pd.read_csv("impact.csv") 
y = impact[['Length.of.stay']]
X = impact.iloc[:,1:]
X = X.drop(['Length.of.stay'],axis = 1)

X

,Age,BloodStool,MucoidStool,NumberDiarEpi,AbdominalPain,ThreeDaysFever,coinf_bi,single_inf_bi,between_common,between_commo_other
0,89,2,2,7.96793,1,2,0,0,0,0
1,1,2,1,7.96793,2,1,1,0,0,1
2,29,2,2,7.96793,1,2,0,0,0,0
3,0,2,1,7.96793,2,2,0,0,0,0
4,1,2,1,7.96793,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
702,30,2,2,6.00000,1,1,0,1,0,0
703,0,2,2,8.00000,2,1,1,0,1,0
704,0,2,2,15.00000,2,2,1,0,0,1
705,1,2,1,9.00000,2,2,1,0,0,1


In [38]:
#x
kernel_ridge = sklearn.model_selection.GridSearchCV(KernelRidge(kernel='rbf'),
                 param_grid={"alpha": np.logspace(-5, 5, num=10),
                             "gamma": np.logspace(-5, 5, num=10)},
                 scoring = 'neg_mean_squared_error', cv=5)
kernel_ridge = kernel_ridge.fit(X,y)

kernel_ridge.best_params_

{'alpha': 0.021544346900318846, 'gamma': 0.0001291549665014884}

In [39]:
#To check the coef of this model
kernel_ridge_coef=np.dot(X.transpose(),
                   KernelRidge(kernel='rbf', 
                               alpha=0.021544346900318846, 
                               gamma=0.0001291549665014884).fit(X, y).dual_coef_)

kr_coef = pd.DataFrame(np.round(kernel_ridge_coef, 4),  
                          X.columns, columns = ["kernel_ridge_coefficients"])
## kr_coef["Exponential of kernel ridge Coefficients"]=np.round(np.exp(kernel_coef), 4)
## kr_coef = kr_coef[kr_coef["Kenel ridge Coefficients"] != 0]
kr_coef = kr_coef.sort_values(by = "kernel_ridge_coefficients", 
ascending = False)
kr_coef

,kernel_ridge_coefficients
MucoidStool,1404.2627
AbdominalPain,595.8394
single_inf_bi,423.9648
between_common,284.0150
NumberDiarEpi,147.4070
coinf_bi,-112.3157
Age,-145.1999
BloodStool,-379.3364
between_commo_other,-396.3308
ThreeDaysFever,-1310.3885


# 